# DINO Animal Detection - Zapovednik

**Шаги:**
1. Запустите ячейки 1-3 (установка)
2. Загрузите датасет (ячейка 4)
3. Запустите анализ (ячейка 5)

In [8]:
!git clone https://github.com/YanGaev2/dino-detection.git
%cd dino-detection
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt
!pip install -r DINO/requirements.txt
print("[OK] Установка завершена")

Cloning into 'dino-detection'...
remote: Enumerating objects: 3089, done.
remote: Total 3089 (delta 0), reused 0 (delta 0), pack-reused 3089 (from 2)
Receiving objects: 100% (3089/3089), 75.22 MiB | 16.74 MiB/s, done.
Resolving deltas: 100% (42/42), done.
/content/dino-detection/dino-detection
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/MultiScaleDeformableAttention-1.0-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/MultiScaleDeformableAttention-1.0-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
[OK] Установка завершена


In [12]:
#@title 2. Скачивание чекпоинта
import os
os.makedirs("output_dino", exist_ok=True)
!curl -L "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Qer6MIgthiA7Ww" -o output_dino/checkpoint.pth
size_mb = os.path.getsize("output_dino/checkpoint.pth") / (1024*1024)
print(f"[OK] Чекпоинт: {size_mb:.1f} MB")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 2489M  100 2489M    0     0  13.4M      0  0:03:05  0:03:05 --:--:-- 13.5M
[OK] Чекпоинт: 2489.9 MB


In [6]:
#@title 3. Сборка CUDA
%cd DINO/models/dino/ops
!python setup.py build install
%cd ../../../..
print("[OK] CUDA собран")

/content/dino-detection/DINO/models/dino/ops
running build
running build_py
creating build/lib.linux-x86_64-cpython-312/modules
copying modules/ms_deform_attn.py -> build/lib.linux-x86_64-cpython-312/modules
copying modules/__init__.py -> build/lib.linux-x86_64-cpython-312/modules
creating build/lib.linux-x86_64-cpython-312/functions
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-cpython-312/functions
copying functions/__init__.py -> build/lib.linux-x86_64-cpython-312/functions
running build_ext
W1227 22:51:36.786000 6489 torch/utils/cpp_extension.py:521] The detected CUDA version (12.5) has a minor version mismatch with the version that was used to compile PyTorch (12.6). Most likely this shouldn't be a problem.
W1227 22:51:36.786000 6489 torch/utils/cpp_extension.py:531] There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.5
building 'MultiScaleDeformableAttention' extension
creating /content/dino-detection/DINO/models/dino/ops/build/temp.l

In [4]:
#@title 4. Загрузка датасета
from google.colab import files
import zipfile
import os
import shutil

print("Загрузите zip архив с датасетом:")
uploaded = files.upload()

for fn in uploaded.keys():
    print(f"Распаковка {fn}...")
    with zipfile.ZipFile(fn, 'r') as z:
        z.extractall('dataset_temp')

# Исправляем Windows пути
os.makedirs('dataset/images', exist_ok=True)
os.makedirs('dataset/labels', exist_ok=True)

for item in os.listdir('dataset_temp'):
    src = os.path.join('dataset_temp', item)
    if '\\' in item:
        # Windows путь в имени файла
        parts = item.split('\\')
        folder = parts[0]
        filename = parts[-1]
        if folder == 'images':
            shutil.copy(src, os.path.join('dataset/images', filename))
        elif folder == 'labels':
            shutil.copy(src, os.path.join('dataset/labels', filename))
    elif item == 'data.yaml':
        shutil.copy(src, 'dataset/data.yaml')
    elif item.endswith(('.jpg', '.png', '.jpeg')):
        shutil.copy(src, os.path.join('dataset/images', item))
    elif item.endswith('.txt'):
        shutil.copy(src, os.path.join('dataset/labels', item))

shutil.rmtree('dataset_temp')

print("[OK] Датасет распакован:")
print(f"  Изображений: {len(os.listdir('dataset/images'))}")
print(f"  Меток: {len(os.listdir('dataset/labels'))}")

Загрузите zip архив с датасетом:


Saving yolo_test_dataset.zip to yolo_test_dataset (1).zip
Распаковка yolo_test_dataset (1).zip...
[OK] Датасет распакован:
  Изображений: 30
  Меток: 30


In [16]:
#@title 5. Запуск анализа
%cd /content/dino-detection

#@markdown **Формат аннотаций:**
dataset_format = "yolo" #@param ["coco", "yolo", "auto"]
#@markdown **Папка с изображениями:**
images_dir = "dataset/images" #@param {type:"string"}
#@markdown **Папка/файл аннотаций (для YOLO - labels, для COCO - JSON):**
annotations = "dataset/labels" #@param {type:"string"}
#@markdown **Порог уверенности:**
threshold = 0.3 #@param {type:"slider", min:0.1, max:0.9, step:0.05}

# Показать структуру датасета
print("Структура датасета:")
!ls -la dataset/
print()

# Запуск анализа
print("Запуск анализа...")
!python run_analysis.py \
    --checkpoint output_dino/checkpoint.pth \
    --images_dir {images_dir} \
    --annotations {annotations} \
    --format {dataset_format} \
    --conf_threshold {threshold}

/content/dino-detection
Структура датасета:
total 20
drwxr-xr-x 4 root root 4096 Dec 27 22:51 .
drwxr-xr-x 7 root root 4096 Dec 27 22:55 ..
-rw-r--r-- 1 root root  250 Dec 27 22:51 data.yaml
drwxr-xr-x 2 root root 4096 Dec 27 22:51 images
drwxr-xr-x 2 root root 4096 Dec 27 22:51 labels

Запуск анализа...
  АНАЛИЗ МОДЕЛИ DINO ДЛЯ ДЕТЕКЦИИ ЖИВОТНЫХ
Дата: 2025-12-27 23:07:55
Checkpoint: output_dino/checkpoint.pth
Images: dataset/images
Annotations: dataset/labels
Confidence threshold: 0.3
OCR: Включен

[ШАГ 1/4] Загрузка модели...
[MODEL] Загрузка конфигурации...
[MODEL] Построение модели...
use_checkpoint!!!!!!!!!!!!!!!!!!!!!!!!
[MODEL] Загрузка весов из output_dino/checkpoint.pth...
[MODEL] Модель загружена (эпоха: 32)

[ШАГ 2/4] Поиск изображений...
[INFO] Найдено 30 изображений
[ANNOTATIONS] Загрузка YOLO аннотаций из dataset/labels...
[ANNOTATIONS] Найдено 30 файлов меток
[ANNOTATIONS] Загружено 30 YOLO аннотаций для 30 изображений

[ШАГ 3/4] Инференс...
[OCR] Инициализация EasyOCR..

In [17]:
#@title 6. Просмотр результатов
import pandas as pd
import os

if not os.path.exists("results/summary_by_species.csv"):
    print("ОШИБКА: Результаты не найдены!")
    print("Убедитесь, что ячейка 5 выполнилась успешно.")
    print("\nПроверьте:")
    print("1. Правильный путь к изображениям (images_dir)")
    print("2. Правильный путь к аннотациям (annotations)")
    print("3. Правильный формат (yolo/coco)")
else:
    print("=" * 60)
    print("СВОДНАЯ ТАБЛИЦА ПО ВИДАМ")
    print("=" * 60)
    display(pd.read_csv("results/summary_by_species.csv"))
    print()
    print("=" * 60)
    print("ДЕТАЛЬНЫЙ ЛОГ (первые 20)")
    print("=" * 60)
    display(pd.read_csv("results/detailed_detections.csv").head(20))

СВОДНАЯ ТАБЛИЦА ПО ВИДАМ


,Вид (EN),Вид (RU),Количество особей
0,mountain_hare,Заяц-беляк,5
1,moose,Лось,14
2,bear,Медведь,3
3,black_grouse,Тетерев,3
4,TOTAL,ИТОГО,25



ДЕТАЛЬНЫЙ ЛОГ (первые 20)


,file_name,class_name,class_name_ru,confidence,date,time,temperature,camera_id,bbox
0,frame_0001_4_2.png,mountain_hare,Заяц-беляк,0.8451,2025-03-16,21:12:34,NaN,NaN,"[1481.1160888671875, 705.6346435546875, 64.558..."
1,frame_0002_4_1.png,mountain_hare,Заяц-беляк,0.8655,2025-02-18,23:29:56,NaN,NaN,"[1344.14599609375, 695.6115112304688, 115.5162..."
2,frame_0004_1_1.png,moose,Лось,0.9092,2025-02-26,06:15:48,NaN,NaN,"[360.6424560546875, 268.31866455078125, 618.12..."
3,frame_0006_9.png,moose,Лось,0.8147,2025-07-13,21:12:39,NaN,NaN,"[593.1925048828125, 14.323639869689941, 758.31..."
4,frame_000910_1.png,mountain_hare,Заяц-беляк,0.7727,2025-02-07,21:31:04,NaN,NaN,"[730.5900268554688, 606.7318115234375, 82.1532..."
5,frame_000960_1.png,bear,Медведь,0.9638,2025-07-05,01:26:00,NaN,NaN,"[1401.0947265625, 627.84130859375, 339.7678222..."
6,frame_001010_2.png,bear,Медведь,0.9584,2025-07-05,NaN,NaN,NaN,"[1401.3616943359375, 621.1637573242188, 363.52..."
7,frame_0010_14_1.png,moose,Лось,0.7850,2025-02-26,06:16:06,NaN,NaN,"[1331.692138671875, 472.038330078125, 197.1235..."
8,frame_0010_1_1.png,moose,Лось,0.8668,2025-02-26,06:15:50,NaN,NaN,"[731.1029663085938, 308.3120422363281, 459.508..."
9,frame_001390_2.png,bear,Медведь,0.9407,2025-07-05,11:01:41,NaN,NaN,"[1617.130615234375, 575.5965576171875, 302.677..."


In [ ]:
#@title 7. Скачать результаты
from google.colab import files
import subprocess
subprocess.run("zip -r results.zip results/", shell=True)
files.download('results.zip')
print("[OK] Скачивание завершено")